In [2]:
import logging
import multiprocessing
from collections import defaultdict

In [3]:
class Node():
    def __init__(self, item, count=0):
        self.item = item
        self.count = count
        self.parent = None
        self.children = {}  # dict (item, Node)

    def __str__(self):
        if self.item is not None:
            s = 'item: {} count: {}  '.format(self.item, self.count)
        else:
            s = 'root \n'
        s += 'children: '
        for child in self.children:
            s += str(child) + ' '

        return s

In [4]:
class FPTree():
    def __init__(self):
        # dict[item] = [Node1, Node2...]
        # header_table dict stores a linked list for every item that has minimum
        #   support so that we can use it for big itemset generation later.
        self.header_table = {}
        self.item_counter = {}
        self.root = Node(None)

    def add_tran(self, tran, weight=1):
        ptr = self.root
        for item in tran:
            if item in ptr.children:
                ptr.children[item].count += weight
                self.item_counter[item] += weight
                # we move down to the current node because the descending ordering
                #   forces next item from the transaction to be placed beneath it.
                ptr = ptr.children[item]
            else:
                new_node = Node(item, weight)
                new_node.parent = ptr
                ptr.children[item] = new_node
                if item in self.header_table:
                    self.header_table[item].append(new_node)
                    self.item_counter[item] += weight
                else:
                    self.header_table[item] = [new_node]
                    self.item_counter[item] = weight
                ptr = new_node

        return

    def mine(self, min_cnt=1): 
        """
        return:
            [list of frequent patterns, list of fp count]
        """
        fp, fp_count = [], []
        for item in self.header_table:
            if self.item_counter[item] >= min_cnt:
                
                # FIRST PART: LOOKING ON ORIGINAL TREE
                fp.append([item])
                fp_count.append(self.item_counter[item]) 
                
                # SECOND PART: LOOKING IN CONDITIONAL TREE (RECURSIVELY)
                cond_trans, weights = self.get_conditional_tran(item, min_cnt)
                cond_tree = FPTree()
                for tran, weight in zip(cond_trans, weights):
                    assert item not in tran, (item, tran, weight)
                    cond_tree.add_tran(tran, weight)
                cond_fp, cond_fp_count = cond_tree.mine(min_cnt)
                if cond_fp: # Append the item that the tree is conditioned upon.
                    cond_fp = [i + [item] for i in cond_fp]
                    fp += cond_fp   # Append the produced item list in the original
                    fp_count += cond_fp_count

        assert len(fp) == len(fp_count) # just a check 
        if fp:
            fp = [sorted(i) for i in fp]
            tmp = list(zip(fp, fp_count))
            tmp = sorted(tmp, key=lambda x: (len(x[0]), x[0]))
            fp, fp_count = list(zip(*tmp))

        return fp, fp_count

    def get_conditional_tran(self, item, min_cnt=1):
        """
        item: excluding item
        return:
        [list of items, list of weight]
        """
        trans, weights = [], []
        for node in self.header_table[item]:
            # if node.count >= min_cnt:
            tmp_tran = []
            ptr = node.parent
            # while ptr.item: // wrong when item == 0
            while ptr.item != None:
                tmp_tran.append(ptr.item)
                ptr = ptr.parent
            if tmp_tran:
                trans.append(tmp_tran.reverse())
                weights.append(node.count)

        return trans, weights

    def print_tree(self):
        l = [self.root]
        while l:
            next_l = []
            for node in l:
                print(node)
                next_l += node.children.values()
            l = next_l
            print('----------------------------------') 

## Testing the construction and mining process

In [37]:
import pandas as pd
from pprint import pprint

df = pd.read_csv('weather.csv', dtype={'Windy': str})

transactions = df.values.tolist()
# count and sort
counter = defaultdict(int)
min_cnt = 6

#     if n_jobs == 1:
if True:
    for tran in transactions:
        for item in tran:
            counter[item] += 1
frequent_item = [(item, counter[item]) for item in counter if counter[item] >= min_cnt]
supported_items = list(zip(*frequent_item))[0]
supported_list = [[item for item in tran if item in supported_items] for tran in transactions]


horizontally_sorted_trans = [sorted(tran, key=lambda x: (counter[x], x), reverse=True) for tran in supported_list]
# vertical sort happens by the sum of the values of each transaction
tran_sum = [sum([counter[item] for item in tran]) for tran in horizontally_sorted_trans]

combined_data = list(zip(tran_sum, horizontally_sorted_trans))
sorted_data = sorted(combined_data, reverse=True)
sorted_values, sorted_transactions = zip(*sorted_data)

pprint(sorted_transactions)

tree = FPTree()
for tran in sorted_transactions:
    tree.add_tran(tran)
tree.print_tree()
tree.mine(6)

(['Yes', 'False', 'Normal', 'Mild'],
 ['Yes', 'False', 'High', 'Mild'],
 ['Yes', 'Normal', 'True', 'Mild'],
 ['Yes', 'High', 'True', 'Mild'],
 ['Yes', 'False', 'Normal'],
 ['Yes', 'False', 'Normal'],
 ['Yes', 'False', 'Normal'],
 ['Yes', 'False', 'High'],
 ['Yes', 'Normal', 'True'],
 ['False', 'High', 'Mild'],
 ['High', 'True', 'Mild'],
 ['False', 'High'],
 ['Normal', 'True'],
 ['High', 'True'])
root 
children: Yes False High Normal 
----------------------------------
item: Yes count: 9  children: False Normal High 
item: False count: 2  children: High 
item: High count: 2  children: True 
item: Normal count: 1  children: True 
----------------------------------
item: False count: 6  children: Normal High 
item: Normal count: 2  children: True 
item: High count: 1  children: True 
item: High count: 2  children: Mild 
item: True count: 2  children: Mild 
item: True count: 1  children: 
----------------------------------
item: Normal count: 4  children: Mild 
item: High count: 2  childre

((['False'],
  ['High'],
  ['Mild'],
  ['Normal'],
  ['True'],
  ['Yes'],
  ['False', 'Yes'],
  ['Normal', 'Yes']),
 (8, 7, 6, 7, 6, 9, 6, 6))

In [20]:
pprint(sorted_transactions)

(['Yes', 'False', 'Normal', 'Mild'],
 ['Yes', 'False', 'High', 'Mild'],
 ['Yes', 'Normal', 'True', 'Mild'],
 ['Yes', 'High', 'True', 'Mild'],
 ['Yes', 'False', 'Normal'],
 ['Yes', 'False', 'Normal'],
 ['Yes', 'False', 'Normal'],
 ['Yes', 'False', 'High'],
 ['Yes', 'Normal', 'True'],
 ['False', 'High', 'Mild'],
 ['High', 'True', 'Mild'],
 ['False', 'High'],
 ['Normal', 'True'],
 ['High', 'True'])
